# Cui2Vec on MIMIC Notes

This project is to grab the CUIs of each concept identified in a given MIMIC note using [QuickUMLS ](https://github.com/Georgetown-IR-Lab/QuickUMLS) and use the [pre-trained CUI embeddings](https://figshare.com/s/00d69861786cd0156d81) to get an embedding that represents the note for all MIMIC notes.

## Imports and Inits

In [1]:
import pandas as pd
import numpy as np
import pickle
import psycopg2

from pathlib import Path

from QuickUMLS.quickumls import QuickUMLS

In [2]:
PATH = Path('data')
QUMLS_PATH = Path('/storage/UMLS-Storage/quickumls')

## Functions

In [3]:
def total_len(l):
    return sum([len(sl) for sl in l])

## Load and save pre-trained CUI Vectors

In [ ]:
cui_vecs_df = pd.read_csv('/storage/pre-trained/cui2vec_pretrained.csv')

In [ ]:
idx2cui = cui_vecs_df['cui'].to_dict()
cui_vecs = cui_vecs_df.loc[:, cui_vecs_df.columns != 'cui'].values

pickle.dump(idx2cui, open(PATH/'idx2cui.pkl', 'wb'))
np.save(PATH/'cui-vecs.npy', cui_vecs)

## Extract CUIs

In [4]:
idx2cui = pickle.load(open(PATH/'idx2cui.pkl', 'rb'))
cui2idx = {c: i for i, c in idx2cui.items()}

cui_vecs = np.load(PATH/'cui-vecs.npy')

In [5]:
qu = QuickUMLS(QUMLS_PATH, threshold=0.8, similarity_name='cosine')

In [6]:
test_note = "The patient showed symtoms of hypertension. No evidence of heart attack but patient had cardiac arrest. He developed type 2 diabetes. But maybe cardiac arrest later with heart attack."

## Checkpoint

In [7]:
matches = qu.match(test_note, best_match=True, ignore_syntax=False)
l = len(matches)
tl = total_len(matches)
if l:
    al = tl/l
else:
    al = 0
l, tl, al

(9, 27, 3.0)

In [8]:
for cs in matches:
    print(cs[0]['ngram'])

type 2 diabetes
cardiac arrest
cardiac arrest
hypertension
heart attack
heart attack
developed
patient
patient


In [36]:
ngram_counts = {}

for cs in matches:
    ngram = cs[0]['ngram']
    if ngram not in ngram_counts:
        ngram_counts[ngram] = 1
    else:
        ngram_counts[ngram] += 1

ngram_counts

{'type 2 diabetes': 1,
 'cardiac arrest': 2,
 'hypertension': 1,
 'heart attack': 2,
 'developed': 1,
 'patient': 2}

In [41]:
ngram_vecs = {}

for cs in matches:
    n_valid_cuis = 0
#     print(cs[0]['ngram'])
    for c in cs:
#         print(c['ngram'], c['term'], c['cui'], c['cui'] in cui2idx)
        if c['cui'] in cui2idx:
            n_valid_cuis += 1
#     print(n_valid_cuis)
    if n_valid_cuis != 0:
        ngram_vecs[c['ngram']] = True
    else:
        if c['ngram'] in ngram_counts:
            del ngram_counts[c['ngram']]
#         if c['cui'] in cui2idx:
#             n_valid_cuis += 1
#             idx = cui2idx[c['cui']]
#             cui_avg += cui_vecs[idx]
#     if n_valid_cuis:
#         cui_avg /= n_valid_cuis
#         ngram_vecs[cs[0]['ngram']] = cui_avg

In [44]:
ngram_vecs.keys() == ngram_counts.keys()

True

In [42]:
ngram_vecs.keys()

dict_keys(['type 2 diabetes', 'cardiac arrest', 'hypertension', 'heart attack'])

In [43]:
ngram_counts.keys()

dict_keys(['type 2 diabetes', 'cardiac arrest', 'hypertension', 'heart attack'])

In [ ]:
note_vector = np.zeros((500))

In [ ]:
for cs in matches:
    n_valid_cuis = 0
    cui_avg = np.zeros(500)
    for c in cs:
        if c['cui'] in cui2idx:
            n_valid_cuis += 1
            idx = cui2idx[c['cui']]
            cui_avg += cui_vecs[idx]
    if n_valid_cuis:
        cui_avg /= n_valid_cuis
        ngram_vecs[cs[0]['ngram']] = cui_avg

In [ ]:
matches[8]

In [ ]:
test_note = '''
Joseph R. Smith

1234567-8

4/5/2006

REASON FOR VISIT:Here to get a new primary care physician.

HPI: Mr. Smith is a 56-year-old gentleman formally followed at Carolina Premier who presents to obtain a new primary care physician secondary to insurance changes. He has a past medical history significant for a myocardial infarction in 1994. His cholesterol has been fine. Catheterization showed a possible "kink" in one of his vessels and it was thought that he had a possible "eddy" of current which led to a clot. He has been on Coumadin since then as well as a calcium channel blocker with the thought that there may have been a superimposed spasm. He has had several unremarkable stress tests since then. He works out on a Nordic-Track three times a week without any chest pain or shortness of breath. He has a history of possible peptic ulcer disease in 1981. He was treated with H2 blockers and his symptoms resolved. He has never had any bleeding to his knowledge. He had a hernia repair bilaterally in 1989 and surgery for a right knee cyst in 1999, all of which went well. He also has about a year long history of right buttock pain. This happens only when he is sitting for some time and does not change position. It does not happen when he is walking or exercising. He wonders if it might be pyriformis syndrome. If he changes positions frequently or stretches his legs, this seems to help. He has no acute complaints today and is here to get plugged into the system. He does wonder if there is anything else that can be done about his buttock pain.

PAST MEDICAL HISTORY: As above. In addition, he had a flexible sigmoidoscopy in 2003 which was okay.

MEDICATIONS: Tylenol p.r.n., baby aspirin p.o. q.d., Coumadin 3.5 mg p.o. q.d., Adalat Time Released tablets 30 mg one p.o. q.d., multivitamin, glucosamine and chondroitin sulfate.

ALLERGIES: No Known Drug Allergies

FAMILY HISTORY: His mother had diabetes developed at age 55 and coronary artery disease in her mid sixties. His father had CAD as well but not until his 70s. A paternal aunt had breast cancer. There is no history of colon or prostate cancer.

SOCIAL HISTORY: He lives in Durham with his wife and mother. He works for a biotech company. He does not smoke. He drinks two beers per night and reports no trouble with alcohol in the past. No history of drug use.

REVIEW OF SYSTEMS: As per his personal health summary and is significant only for his buttock pain as listed above, but is otherwise essentially unremarkable.

PHYSICAL EXAM:

VITAL SIGNS: Weight 175.2 pounds which is 79.3 kg, blood pressure 142/96 by the nurse, 140/92 by me, pulse is 64.

GENERAL: A healthy-appearing middle-aged gentleman.

HEENT: Pupils equal, round, reactive to light. Conjunctivaepink. Sclerae anicteric. Tympanic membranes clear. Oropharynx clear.

NECK: No lymphadenopathy or thyromegaly or JVD.

LUNGS: Clear to auscultation and percussion.

HEART: Regular rate and rhythm without murmur, rub, or gallop.

ABDOMEN: Normal bowel sounds. Soft, nontender. No hepatosplenomegaly.

EXTREMITIES: No cyanosis, clubbing, or edema. 2+ peripheral pulses.

NEUROLOGIC: Motor and sensation grossly intact.

PSYCHIATRIC: Normal affect and behavior.

DERMATOLOGIC: He has a small whitish papule at the upper borderof his mustache on the left.

MUSCULOSKELETAL: Full range of motion of his legs and hips bilaterally with no tenderness to palpation over his buttocks.

ASSESSMENT/PLAN:

Status post myocardial infarction. No evidence for actual CAD. He will continue his aspirin and Coumadin. Unclear to exactly how long he should be on Coumadin or if this is really needed. At some point, may discuss this with cardiology. We will check an INR today and get him plugged into our Coumadin clinic. Check cholesterol.
High blood pressure. He will continue with diet and exercise changes. At next visit, may go up on his calcium channel blocker versus add another agent. Check creatinine and potassium today.
Buttock pain. Will try some physical therapy.
Will send him to dermatology for the papule above his mustache.
Possible h/o peptic ulcer disease. As he has not had any known GI bleed, do not feel a need to check H. pylori serology today.
Health maintenance. Tetanus shot today. Flex sig as above. Will discuss prostate cancer screening next visit.
Return to clinic in four months.
John Student, MS3

Seen with Joe Doctor, MD
'''

## Grab sample data from MIMIC

In [ ]:
cats = pd.read_csv('cats.csv')
max_limit = 5

queries = []
for category, n_notes in zip(cats['category'], cats['number_of_notes']):
    limit = min(max_limit, n_notes) if max_limit > 0 else n_notes
    if limit == max_limit:
        q = f"""
        select category, text from correctnotes where category=\'{category}\' order by random() limit {limit};
        """
    else:
        q = f"""
        select category, text from correctnotes where category=\'{category}\';
        """
    queries.append(q)

dfs = []

con = psycopg2.connect(dbname='mimic', user='sudarshan', host='/var/run/postgresql')
for q in queries:
    df = pd.read_sql_query(q, con)
    dfs.append(df)
con.close()
    
df = pd.concat(dfs)
# df.set_index('row_id', inplace=True)
df.shape

## pyContext

We need to encode domain knowledge in the modifiers and targets for using pyContext

In [ ]:
import pyConTextNLP.pyConText as pyConText
import pyConTextNLP.itemData as itemData
import networkx as nx

In [ ]:
modifiers = itemData.get_items(
    "https://raw.githubusercontent.com/chapmanbe/pyConTextNLP/master/KB/lexical_kb_05042016.yml")
targets = itemData.get_items(
    "https://raw.githubusercontent.com/chapmanbe/pyConTextNLP/master/KB/utah_crit.yml")

In [ ]:
markup = pyConText.ConTextMarkup()

In [ ]:
markup.setRawText(test_note.lower())
markup.cleanText()
markup.markItems(modifiers, mode='modifier')
markup.markItems(targets, mode='target')

In [ ]:
for node in markup.nodes(data=True):
    print(node)